In [3]:
import pickle
import pandas as pd
import torch

查看原始的数据集是什么样子的

In [4]:
with open("./company_event/train.txt","rb") as f:
    corpus_index_train0=pickle.load(f)

In [4]:
len(corpus_index_train0),corpus_index_train0

(1600,
 [[[('破产', 'None', '发生', '破产清算', '天地科技', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('缺陷', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('整改', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('充公', 'None', '发生', '违法违规', '天地科技', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '天地科技', '客体公司')],
   [('低于', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('调查', 'None', '发生', '违法违规', '新华网', '客体公司'),
    ('调查', 'None', '发生', '负面报道', '沈阳化工', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '天地科技', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '天地科技', '客体公司')],
   4],
  [[('低于', 'None', '发生', '经营异常', '金杯电工', '客体公司'),
    ('充公', 'None', '发生', '违法违规', '金杯电工', '客体公司'),
    ('整改', 'None', '发生', '经营异常', '金杯电工', '客体公司'),
    ('原材料价格上涨', 'None', '发生', '业务异常', '金杯电工', '客体公司'),
    ('传闻', 'None', '发生', '违法违规', '金杯电工', '客体公司'),
    ('低于', 'None', '发生', '经营异常', '金杯电工', '客体公司'),
    ('亏损', 'None', '发生', '业务异常

In [5]:
with open("./company_event/dev.txt","rb") as f:
    corpus_index_dev=pickle.load(f)
with open("./company_event/test.txt","rb") as f:
    corpus_index_test=pickle.load(f)

拿出每个句子的第一个单词并且做成一个字典，每个单词不重复

In [6]:
corpus_list=[]
# 选择处理的数据的源 （训练集、验证集、测试集）
for source in [corpus_index_train0,corpus_index_dev,corpus_index_test]:
    # 遍历所有的序列+候选组合
    for i in range(len(source)):
        # 分别遍历每个组合的序列和候选
        for j in range(2):
            # 遍历序列
            if j==0:
                for k in range(8):
                    corpus_list.append(source[i][j][k][0].replace("'",""))
            # 遍历候选
            if j==1:
                for k in range(5):
                    corpus_list.append(source[i][j][k][0].replace("'",""))
corpus_list[:100]

['破产',
 '低于',
 '缺陷',
 '整改',
 '低于',
 '充公',
 '低于',
 '低于',
 '低于',
 '调查',
 '调查',
 '低于',
 '低于',
 '低于',
 '充公',
 '整改',
 '原材料价格上涨',
 '传闻',
 '低于',
 '亏损',
 '亏损',
 '同比下降',
 '同比下降',
 '同比下降',
 '低于',
 '同比下降',
 '整改',
 '整治',
 '整治',
 '否认',
 '亏损',
 '整改',
 '整治',
 '停产',
 '整改',
 '低于',
 '处罚',
 '限期整改',
 '低于',
 '同比下降',
 '同比下降',
 '举报',
 '调查',
 '净利润下降',
 '同比下降',
 '举报',
 '整顿',
 '起诉',
 '起诉',
 '利润下降',
 '整改',
 '缺少',
 '停产',
 '亏损',
 '低于',
 '低于',
 '同比下降',
 '调查',
 '亏损',
 '低于',
 '利润下降',
 '低于',
 '判决',
 '停产',
 '判决',
 '低于',
 '利润下降',
 '低于',
 '整治',
 '低于',
 '低于',
 '低于',
 '调查',
 '低于',
 '低于',
 '低于',
 '调查',
 '净利润下降',
 '亏损',
 '亏损',
 '亏损',
 '亏损',
 '亏损',
 '低于',
 '亏损',
 '亏损',
 '亏损',
 '否认',
 '否认',
 '亏损',
 '亏损',
 '同比下降',
 '低于',
 '同比下降',
 '低于',
 '同比下降',
 '调查',
 '调查',
 '调查',
 '处罚']

In [7]:
with open("./company_event/corpus_verb_mapping.pkl","wb") as f:
    pickle.dump(sorted(set(corpus_list)),f)
with open("./company_event/corpus_verb_mapping.pkl","rb") as f:
    corpus_verb_mapping=pickle.load(f)
corpus_verb_mapping[:20]

['下架',
 '不合格',
 '不存在',
 '不属实',
 '不得使用',
 '不达标',
 '不达预期',
 '不过关',
 '严重失实',
 '中止重组',
 '举报',
 '亏损',
 '交易量减少',
 '产能过剩',
 '产量减少',
 '人民币升值',
 '以次充好',
 '传言',
 '传闻',
 '伪造']

In [10]:
corpus_verb_mapping.index("伪造"),len(corpus_verb_mapping)

(19, 219)

处理事件之间的序列关系，并且保存

In [11]:
# 选择处理的数据的源 （训练集、验证集、测试集）
for source in [corpus_index_train0,corpus_index_dev,corpus_index_test]:
    event_sequence_list=[]
    # 遍历所有的序列+候选组合
    for i in range(len(source)):
        cur_sequence_list=[]
        # 分别遍历每个组合的序列和候选
        for j in range(2):
            # 遍历序列
            if j==0:
                for k in range(8):
                    cur_sequence_list.append(corpus_verb_mapping.index(source[i][j][k][0].replace("'","")))
            # 遍历候选
            if j==1:
                target=source[i][2]
                cur_sequence_list.append(corpus_verb_mapping.index(source[i][j][target][0].replace("'","")))
        event_sequence_list.append(cur_sequence_list)
    if source==corpus_index_train0:
        with open("./company_event/train_sequence.pkl","wb") as f:
            pickle.dump(event_sequence_list,f)
    elif source==corpus_index_dev:
        with open("./company_event/dev_sequence.pkl","wb") as f:
            pickle.dump(event_sequence_list,f)
    else:
        with open("./company_event/test_sequence.pkl","wb") as f:
            pickle.dump(event_sequence_list,f)

In [12]:
corpus_index_dev

[[[('亏损', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('亏损', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('举报', 'None', '发生', '负面报道', '长园集团', '客体公司'),
   ('举报', 'None', '发生', '负面报道', '长园集团', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('低于', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('净利润下降', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '长园集团', '客体公司')],
  [('低于', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('整改', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('低于', 'None', '发生', '经营异常', '寒锐钴业', '客体公司'),
   ('整改', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('低于', 'None', '发生', '违法违规', '五粮液', '客体公司')],
  0],
 [[('低于', 'None', '发生', '经营异常', '金刚玻璃', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('低于', 'None', '发生', '经营异常', '金刚玻璃', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('资产减值', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('资产减值', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('立案', 'None', '发生', '违法违规', '金刚玻璃', '客体公司')

构建对应的知识图

In [4]:
import dgl
import numpy as np

In [10]:
with open("./company_event/train_sequence.pkl","rb") as f:
    train_sequence=pickle.load(f)
with open("./company_event/dev_sequence.pkl","rb") as f:
    dev_sequence=pickle.load(f)
with open("./company_event/test_sequence.pkl","rb") as f:
    test_sequence=pickle.load(f)

print(len(train_sequence),len(dev_sequence),len(test_sequence))

src_dst=np.empty(shape=(8*(len(train_sequence)+len(dev_sequence)+len(test_sequence)),2),dtype=np.long)
row=0
for source in [train_sequence,dev_sequence,test_sequence]:
    for i in range(len(source)):
        for j in range(8):
            src_dst[row,0]=source[i][j]
            src_dst[row,1]=source[i][j+1]
            row=row+1
np.save("./company_event/src_dst.npy",src_dst)


1600 500 240


C:\Users\Weirdo\AppData\Local\Temp\ipykernel_2524\4062996827.py:10: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  src_dst=np.empty(shape=(8*(len(train_sequence)+len(dev_sequence)+len(test_sequence)),2),dtype=np.long)


保存事件之间的关系图

In [12]:
import torch
src_dst=torch.from_numpy(src_dst)
src=src_dst[:,0]
dst=src_dst[:,1]
# g=dgl.graph(data=(src,dst),num_nodes=len(corpus_verb_mapping))
# dgl.save_graphs("data/dataset/NYT/kg/graph.bin",[g])

构建DGL-KE需要的五个文件，进行transE的训练

In [15]:
import pandas as pd
src_dst_dataframe=pd.DataFrame(columns=["src","dst"],data=src_dst,dtype=int)
src_dst_train=src_dst_dataframe.iloc[:row*2//3,:]
src_dst_valid=src_dst_dataframe.iloc[row*2//3:row*8//9,:]
src_dst_test=src_dst_dataframe.iloc[row*2//9:,:]

In [16]:
src_dst_train.iloc[:5],src_dst_valid[:5],src_dst_test[:5]

(   src  dst
 0  132   20
 1   20  142
 2  142  104
 3  104   20
 4   20   38,
        src  dst
 12480   73  115
 12481  115   20
 12482   20   20
 12483   20   20
 12484   20   20,
       src  dst
 4160   11  160
 4161  160   38
 4162   38  141
 4163  141   73
 4164   73  104)

In [18]:
src_dst_train.to_csv("./company_event/kg/src_dst_train.csv",header=False,index=False)
src_dst_valid.to_csv("./company_event/kg/src_dst_val.csv",header=False,index=False)
src_dst_test.to_csv("./company_event/kg/src_dst_test.csv",header=False,index=False)

In [9]:
verb_dict=dict(zip(range(len(corpus_verb_mapping)),corpus_verb_mapping))
with open("./company_event/verb_dict.dict","w",encoding="utf-8") as f:
    for k,v in verb_dict.items():
        f.write(str(k)+","+v)
        f.write("\n")

In [2]:
import pandas as pd
import pickle
with open("data/dataset/NYT/train_sequence.pkl","rb") as f:
    train_sequence=pickle.load(f)
with open("data/dataset/NYT/dev_sequence.pkl","rb") as f:
    dev_sequence=pickle.load(f)
with open("data/dataset/NYT/test_sequence.pkl","rb") as f:
    test_sequence=pickle.load(f)
src_dst=pd.read_csv("data/dataset/NYT/kg/src_dst.csv")
src_dst_train=src_dst.iloc[:8*len(train_sequence),:]
src_dst_valid=src_dst.iloc[:8*len(dev_sequence),:]
src_dst_test=src_dst.iloc[:8*len(test_sequence),:]
src_dst_train.to_csv("./company_event/src_dst_train.csv",header=False,index=False)
src_dst_valid.to_csv("./company_event/src_dst_val.csv",header=False,index=False)
src_dst_test.to_csv("./company_event/src_dst_test.csv",header=False,index=False)

In [3]:
import pickle
with open("./company_event/dev_sequence.pkl","rb") as f:
    dev_seq=pickle.load(f)

dev_seq[:5]

[[11, 11, 10, 10, 73, 20, 44, 73, 20],
 [20, 73, 20, 73, 168, 168, 137, 47, 73],
 [38, 11, 160, 20, 38, 38, 20, 20, 38],
 [50, 11, 20, 104, 211, 115, 160, 11, 47],
 [11, 11, 11, 11, 160, 11, 11, 170, 11]]